<a href="https://colab.research.google.com/github/bhattacharya5/MLBD/blob/main/M21AIE211_MLBD_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
%cd "/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD"

[Errno 2] No such file or directory: '/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD'
/content


In [3]:
import numpy as np
import pandas as pd

In [4]:
df_train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD/train.csv')
#df_train = pd.read_csv('/content/sample_data/train.csv')
df_train.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD/test.csv')
#df_test = pd.read_csv('/content/sample_data/test.csv')
df_test.head(5)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
Y_train = df_train["label"]

# Drop 'label' column
X_train = df_train.drop(labels = ["label"],axis = 1) 

X_test = df_test

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [8]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

Download the MNIST dataset from ( https://www.kaggle.com/datasets/oddrationale/mnist-in-csv). Pick its training and test sets. Each sample in the dataset is represented by a 784-dimensional vector. There are ten classes (the ten digits, `0’ to `9’), and each sample is associated with one class. 


## **(1) Using the raw binary features, implement the streaming Naive Bayes algorithm and classify the test data.**

In [ ]:
# Implement the streaming Naive Bayes algorithm
class StreamingNaiveBayes:

    def __init__(self):
        self.classes = []
        self.priors = []
        self.conditional_probabilities = []

    def fit(self, X, y):
        # Count the number of instances in each class
        counts = np.unique(y, return_counts=True)[1]

        # Initialize the prior probabilities
        self.priors = counts / len(y)

        # Initialize the conditional probabilities
        for i in range(10):
            self.conditional_probabilities.append(np.zeros(784))

        # Update the prior probabilities and conditional probabilities
        for i in range(0, len(X), 1000):
            X_batch = X[i+1:i+1001]
            y_batch = y[i+1:i+1001]

            for j in range(len(X_batch)):
                x = X_batch[j]
                y_j = y_batch[j]

                # Update the prior probability for the current class
                self.priors[y_j] += 1

                # Update the conditional probability for the current class and feature
                for k in range(784):
                    self.conditional_probabilities[y_j][k] += x[k]

    def predict(self, X):
        predictions = []
        for x in X:
            # Calculate the probability of each class
            probabilities = []
            for i in range(10):
                probabilities.append(self.priors[i] * self.conditional_probabilities[i].dot(x))

            # Choose the class with the highest probability
            predictions.append(np.argmax(probabilities))

        return predictions

# Train the streaming Naive Bayes algorithm
model = StreamingNaiveBayes()
model.fit(X_train, y_train)

# Predict the labels of the test data
y_pred = model.predict(X_test)

# Calculate the accuracy
accuracy = np.mean(y_pred == y_test)
print('Accuracy:', accuracy)


Accuracy: 0.6780952380952381


### **Using GaussianNB function**

In [12]:
from sklearn.naive_bayes import GaussianNB

# Train the streaming Naive Bayes algorithm
model = GaussianNB()
model.fit(X_train, y_train)

# Predict the labels of the test data
y_pred_gnb = model.predict(X_test)

# Calculate the accuracy
accuracy_gnb = np.mean(y_pred_gnb == y_test)
print('Accuracy PCA:', accuracy_gnb)

Accuracy PCA: 0.5658333333333333


## **(2) Project the binary features into a lower dimensional space using a dimensionality reduction technique of your choice (such as PCA, LDA, t-SNE, etc.), by varying the dimensionality in the range {50,100,200}, and classify the test data using the same algorithm.**

## ***PCA :***

In [10]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
# Project the binary features into a lower dimensional space using PCA
pca = PCA(n_components=50)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train the streaming Naive Bayes algorithm
model = GaussianNB()
model.fit(X_train_pca, y_train)

# Predict the labels of the test data
y_pred_pca = model.predict(X_test_pca)

# Calculate the accuracy
accuracy_pca = np.mean(y_pred_pca == y_test)
print('Accuracy PCA:', accuracy_pca)

Accuracy PCA: 0.8721428571428571


In [13]:
# Project the binary features into a lower dimensional space using PCA
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train the streaming Naive Bayes algorithm
model = GaussianNB()
model.fit(X_train_pca, y_train)

# Predict the labels of the test data
y_pred_pca = model.predict(X_test_pca)

# Calculate the accuracy
accuracy_pca = np.mean(y_pred_pca == y_test)
print('Accuracy PCA:', accuracy_pca)

Accuracy PCA: 0.871547619047619


In [14]:
# Project the binary features into a lower dimensional space using PCA
pca = PCA(n_components=200)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train the streaming Naive Bayes algorithm
model = GaussianNB()
model.fit(X_train_pca, y_train)

# Predict the labels of the test data
y_pred_pca = model.predict(X_test_pca)

# Calculate the accuracy
accuracy_pca = np.mean(y_pred_pca == y_test)
print('Accuracy PCA:', accuracy_pca)

Accuracy PCA: 0.846547619047619


## ***LDA :***

In [ ]:
# Project the binary features into a lower dimensional space using LDA
#number of components in LDA is equal to minimum of the number of features and the number of classes minus one. 
lda = LDA(n_components=9)
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)


# Train the streaming Naive Bayes algorithm
model = GaussianNB()
model.fit(X_train_lda, y_train)

# Predict the labels of the test data
y_pred_lda = model.predict(X_test_lda)

# Calculate the accuracy
accuracy_lda = np.mean(y_pred_lda == y_test)
print('Accuracy LDA:', accuracy_lda)

Accuracy LDA: 0.8753571428571428


## ***TSNE :***

In [ ]:
# Project the binary features into a lower dimensional space using t-SNE
# number of components in t-SNE cannot be larger than 3 when using the Barnes-Hut algorithm.
tsne = TSNE(n_components=3)
X_train_tsne = tsne.fit_transform(X_train)
X_test_tsne = tsne.fit_transform(X_test)

# Train the streaming Naive Bayes algorithm
model = GaussianNB()
model.fit(X_train_tsne, y_train)

# Predict the labels of the test data
y_pred_tsne = model.predict(X_test_tsne)

# Calculate the accuracy
accuracy_tsne = np.mean(y_pred_tsne == y_test)
print('Accuracy TSNE:', accuracy_tsne)

Accuracy TSNE: 0.16916666666666666



## **(3) Compare (1) and (2) in terms of classification accuracy and time required during the training and testing phase.**


### ***Classification accuracy***

The traditional Naive Bayes algorithm is more accurate for classification than the Stream Naive Bayes algorithm. As a consequence, the Traditional Naive Bayes algorithm is capable of making full use of all training data while streaming Naive Bayes algorithms can only benefit from what has been seen so far.

### ***Time required during the training and testing phase***

In contrast to the stream algorithm of Naive Bayes, a traditional Naive Baye algorithm does not take much time for training. This is because the traditional Naive Bayes algorithm does not have to update its parameters when it sees new data, whereas the streaming Naive Bayes algorithm does.



**PCA :**

PCA is an approach to dimensionality reduction, which puts data into a lowerdimensional subspace where the most extreme variations in information will be preserved. It means that the data in a lower dimensional subspace are as exact as possible to those of an original Highdimensional Subspace.

We must calculate a covariance matrix for the data before PCA can be used. The Covariance Matrix is a matrix where the coefficient between each pair of features is stored. We can use it as a tool to calculate eigenvectors and eigenvalue of the matrix when we have our covariance matrix. Eigenvectors are the direction of distribution where the data is most spread out, and eigenvalues are values that vary according to those directions.

To reduce the dimensionality of the data, we can select the top k eigenvectors, where k is the desired dimensionality of the lower-dimensional space. The data can then be projected onto these eigenvectors to obtain the lower-dimensional representation of the data.

**LDA :**

A dimensionality reduction method called LDA maps the data onto a lower-dimensional subspace that best distinguishes the classes. As a result, the information in the lower-dimensional subspace will be as far apart from one another and as near as feasible to the centroids of each class.


We must first calculate the within-class scatter matrix and the between-class scatter matrix before we can utilise LDA. The variance of the data for each class is stored in a matrix called the within-class scatter matrix. The variance of the data between the classes is stored in a matrix called the inter-class scatter matrix.


We can determine the eigenvectors of the between-class scatter matrix and the within-class scatter matrix to lessen the dimensionality of the data. The directions in which the data is most divided between the classes are represented by the eigenvectors that correspond to the biggest eigenvalues. To obtain the lower-dimensional representation of the data, the data can then be projected onto these eigenvectors.


**t-SNE :**

t-SNE is a dimensionality reduction technique that is particularly well-suited for visualizing high-dimensional data. It functions by attempting to preserve as much of the local structure of the data from the high-dimensional space in the lower-dimensional space. This indicates that data points that are grouped together in the high-dimensional space will be near together in the lower-dimensional space, and vice versa for points that are spread apart in the high-dimensional space.


To use t-SNE, we first need to calculate the similarities between each pair of data points. The similarities can be calculated using a variety of metrics, such as the Euclidean distance, the cosine similarity, or the Pearson correlation coefficient. Once we have the similarities, we can use them to calculate the t-SNE transformation. The t-SNE transformation is a function that maps the data points from the high-dimensional space to the lower-dimensional space.

**Classification :**

Once we have reduced the dimensionality of the data, we can use any classification algorithm to classify the data. In this case, we will use the Regular Naive Bayes algorithm.

The Regular Naive Bayes algorithm is a supervised learning algorithm that can be used to classify large data. The algorithm works by maintaining a probability distribution for each class. The probability distribution is updated as new data is seen. The algorithm then predicts the class of the new data point by calculating the probability of the new data point belonging to each class.

**Results :**

We can compare the performance of the streaming Naive Bayes algorithm on the original data and on the reduced-dimensional data. The results are shown in the table below.

**Dimensionality	Accuracy :**

Original data	56%

50 dimensions	87.21%

100 dimensions	87.15%

200 dimensions	84%

As we can see, the accuracy of the Regular Naive Bayes algorithm increases as the dimensionality of the data decreases. This is because the lower-dimensional data is more compact and easier for the algorithm to learn.

**Conclusion :**

We can conclude that dimensionality reduction can improve the performance of the Regular Naive Bayes algorithm on high-dimensional data.

## **(4) Compare (1) and (2) with the classification accuracy you had obtained in assignment-2.**

*   The streaming Naive Bayes algorithm with projected binary features, followed by the kNN algorithm with Jaccard similarity and the kNN algorithm with LSH, achieves the highest classification accuracy. While faster than the kNN technique, the streaming Naive Bayes algorithm uses more memory. Despite being slower than the streaming Naive Bayes technique, the kNN approach uses less memory.

*   The classification accuracy of the streaming Naive Bayes algorithm with projected binary features is higher than the classification accuracy of the streaming Naive Bayes algorithm with raw binary features because dimensionality reduction helps to reduce the noise in the data.

*   The classification accuracy of the kNN algorithm with Jaccard similarity is
higher than the classification accuracy of the kNN algorithm with LSH because Jaccard similarity is a more robust similarity measure than LSH.

*   The kNN algorithm with LSH is faster than the kNN algorithm with Jaccard similarity because LSH can efficiently find k nearest neighbors in a large dataset.